# GT98 revisited


In [9]:
using ModelingToolkit, DomainSets

In [29]:

@variables x t f(x,t) Δx
∂ₓ=Differential(x)
∂ₜ=Differential(t)

(::Differential) (generic function with 2 methods)

In [25]:
g(x,t)=1//2*x^2+1//6*x^3 +x*t

g (generic function with 1 method)

In [26]:
h(x,t)=expand_derivatives(∂ₓ(g(x,t)))

h (generic function with 1 method)

In [31]:
g(Δx,t)

(1//6)*(Δx^3) + (1//2)*(Δx^2) + t*Δx

In [32]:
h(x,t)

t + x + (1//2)*(x^2)

In [33]:
h(Δx,t)

0

In [17]:
Ix=Integral(x in DomainSets.ClosedInterval(-Inf,+Inf))

(::Integral{Symbolics.VarDomainPairing}) (generic function with 2 methods)

In [21]:
H(x,t)=expand(Ix(h(x,t)))

H (generic function with 1 method)

In [22]:
H(Δx,t)

Integral(x, -Inf..Inf)(0)

In [ ]:
# SymbolicNumericIntegration usage 
@variables x a b xa xb
f(x,a,b), dummy, dummy2=integrate(cosh(a * x) * exp(b * x), x; symbolic = true)
f(x,xa,xb)

We are looking for the expression for 
\begin{equation}
\int dt \psi^{(n)} \int dx \phi^{(m)} \left [ \rho (x) \frac{\partial^2}{\partial t^2} u(x,t) - \frac{\partial}{\partial x} \left ( \mu (x) \frac{\partial}{\partial x} u(x,t) \right ) \right ]
\end{equation}
as the inner product:

\begin{equation}
\sum_{m^\prime = m-1}^{m+1} \sum_{n^\prime = n-1}^{n+1} A_{m^\prime n^\prime} u (x_{m^\prime}, t_{n^\prime})
\end{equation}



We use this integration by parts in order to prepare the spatial and temporal integration symbolically.

\begin{equation}
\int_{y_1}^{y_2} dy f(y) w(y) = \left [F(y) w(y) \right ]_{y_1}^{y_2} - \int_{y_1}^{y_2} dy \frac{\partial w(y)}{\partial y} F(y)
\end{equation}
with
\begin{equation}
F(y)=\int dy f(y)
\end{equation}
Since $F(y)$ can be obtained analytically with the aid of Taylor expansion, this symbolic computation is feasible.

If we use spline functions, the above equation can be further developed as:
\begin{align}
\int_{y_1}^{y_2} dy f(y) w(y) &=& \left [F^{(1)}(y) w(y) \right ]_{y_1}^{y_2} - \int_{y_1}^{y_2} dy  F^{(1)}(y) \frac{\partial w(y)}{\partial y} \\
&=& \left [F^{(1)}(y) w(y) \right ]_{y_1}^{y_2} -  \left [F^{(2)}(y) \frac{\partial w(y)}{\partial y} \right ]_{y_1}^{y_2} + \int_{y_1}^{y_2} dy F^{(2)}(y) \frac{\partial^2 w(y)}{\partial y^2} \\
&=& \left [F^{(1)}(y) w(y) \right ]_{y_1}^{y_2} -  \left [F^{(2)}(y) \frac{\partial w(y)}{\partial y} \right ]_{y_1}^{y_2} + \left [F^{(3)}(y) \frac{\partial^2 w(y)}{\partial y^2} \right ]_{y_1}^{y_2} - \cdots 
\end{align}

# specific simplification
Apart from the edges, normally we (prefer to) use weighting functions that have zero at the end point/curve/surfaces. This will finally make the above expression simpler, if $w$ is $N$-times derivable.

\begin{equation}
\int_{y_1}^{y_2} dy f(y) w(y) = \sum_{i=1 \ \textrm{or} \ 2}^{N} (-1)^{(i-1)}\left [ F^{(i)}(y) \frac{\partial^{(i-1)}}{\partial y^{(i-1)}}w(y)\right ]_{y_1}^{y_2}
\end{equation}


In [ ]:
orderUtime=2
orderUspace=2
#m = 15 # the midpoint to evaluate the operator at
L₁ₗ = 1 # the leftmost point for x1 direction
L₁ᵣ = 1 # the rightmost point
Lₜₗ = 1 # the leftmost point for t direction
Lₜᵣ = 1 # the rightmost point


orderStructure=L₁ₗ+L₁ᵣ # number of nodes minus one

@variables dx₁ dt   
@variables Cᴾ[1:orderStructure+1] Cᴹ[1:orderStructure+1] Cᵘ[1:orderUspace+1,1:orderUtime+1] # These are the derivatives at the node point xₘ, tₙ
#@variables x¹ t




#@variables ξ¹[1:L₁ₗ+L₁ᵣ+1]  τ[1:Lₜₗ+Lₜᵣ+1] # These are the node points in the spatial and time domain
#@variables ξ¹ₘ[-L₁ₗ:L₁ᵣ] τₙ[-L₁ₗ:L₁ᵣ] ρₘ[-L₁ₗ:L₁ᵣ] μₘ[-L₁ₗ:L₁ᵣ]
ξ¹ₘ = Symbolics.variables(:ξ¹ₘ,-L₁ₗ:L₁ᵣ) # nodes in space (x₁ direction)
τₙ = Symbolics.variables(:τₙ,-Lₜₗ:Lₜᵣ) # nodes in time
ρₘ = Symbolics.variables(:ρₘ,-L₁ₗ:L₁ᵣ)
μₘ = Symbolics.variables(:μₘ,-L₁ₗ:L₁ᵣ)
uₘₙ= Symbolics.variables(:uₘₙ,-L₁ₗ:L₁ᵣ,-Lₜₗ:Lₜᵣ)

ρ(dx₁) = sum(Cᴾ[i] * dx₁^(i-1) / factorial(BigInt(i-1)) for i in 1:orderStructure+1)
μ(dx₁) = sum(Cᴹ[i] * dx₁^(i-1) / factorial(BigInt(i-1)) for i in 1:orderStructure+1)
u(dx₁,dt) = sum(sum(Cᵘ[i,j] * dx₁^(i-1) * dt^(j-1) / factorial(BigInt(i-1)) / factorial(BigInt(j-1)) for i in 1:orderUspace+1) for j in 1:orderUtime+1)

## First step
We need to construct the Taylor expansion coefficients for $\rho$ and $\mu$ from nodes

In [ ]:
# dummy X and T for X = x-xₘ T = t-tₘ
@variables X,T # (continuous) x and t from xₘ and tₙ
# define derivatives
DT=Differential(T)
DX=Differential(X)
DT2=DT^2
DX2=DX^2

In [ ]:
ρ(ξ¹ₘ[2]-ξ¹ₘ[1])

If we use an arregular mesh:

In [ ]:
# This should be used for an irregular mesh
eqsρ=[]
eqsμ=[]
eqsu=[]
for i¹ in 1:L₁ₗ+L₁ᵣ+1
    dx¹=ξ¹ₘ[i¹]-ξ¹ₘ[L₁ₗ+1]
    eqsρ = push!(eqsρ, ρₘ[i¹]~ ρ(dx¹))
    eqsμ = push!(eqsμ, μₘ[i¹]~ μ(dx¹))
    for iᵗ in 1:Lₜₗ+Lₜᵣ+1
        dt = τₙ[iᵗ]-τₙ[Lₜₗ+1]
        eqsu = push!(eqsu, uₘₙ[i¹,iᵗ]~u(dx¹,dt))
    end
end



If we use a regular mesh:


In [ ]:
# Here we can also think about a regular mesh
@variables dx Δx dt Δt
eqsρ=[]
eqsμ=[]
eqsu=[]
for i¹ in 1:L₁ₗ+L₁ᵣ+1
    dx¹=Δx*(i¹-L₁ₗ-1)
    eqsρ = push!(eqsρ, ρₘ[i¹]~ ρ(dx¹))
    eqsμ = push!(eqsμ, μₘ[i¹]~ μ(dx¹))
    for iᵗ in 1:Lₜₗ+Lₜᵣ+1
        dt = Δt*(iᵗ-Lₜₗ-1)
        eqsu = push!(eqsu, uₘₙ[i¹,iᵗ]~u(dx¹,dt))
    end
end

In [ ]:
newCᴾ=Symbolics.solve_for(eqsρ,Cᴾ)
newCᴹ=Symbolics.solve_for(eqsμ,Cᴹ) # if I do simplify here, it doesn't do a good job
newCᵘ=reshape(Symbolics.solve_for(eqsu,Cᵘ),(L₁ₗ+L₁ᵣ+1,Lₜₗ+Lₜᵣ+1))
simplify(newCᴹ[2]) # maybe for the moment simplify should work only for an element
# this newCᴹ and newCᴾ are the derivatives to reconstruct polynomial expressions for continuous ρ and μ

newρ(dx₁) = sum(newCᴾ[i] * dx₁^(i-1) / factorial(BigInt(i-1)) for i in 1:orderStructure+1)
newμ(dx₁) = sum(newCᴹ[i] * dx₁^(i-1) / factorial(BigInt(i-1)) for i in 1:orderStructure+1)
#newu(dx₁,dt) = sum(sum(newCᵘ[i,j] * dx₁^(i-1) * dt^(j-1) / factorial(BigInt(i-1)) / factorial(BigInt(j-1)) for i in 1:orderUspace+1) for j in 1:orderUtime+1)

In [ ]:
newμ(X)
#Symbolics.coeff(X^2+2*X,X)

In [ ]:
simplify(u(X,T))

In [ ]:
#newu(X,T)
# newu is not a good idea 

# Second step

In [ ]:

#u(X,T)
expand_derivatives(Differential(X)(u(X,T)))


In [ ]:
# expression of acceleration:

ρü=newρ(X)* expand_derivatives(DT2(u(X,T)))
expand(ρü)

In [ ]:
# expression of (μu′)′

μu′ = expand(newμ(X)*expand_derivatives(DX(u(X,T))))
kinematics= expand(expand_derivatives(DX(μu′)))

In [ ]:
# here we give the integrand
integrand=ρü-kinematics

In [ ]:
simplify(integrand(X,T))

In [ ]:
simplify(substitute(integrand, Dict(X=>Δx)))

In [ ]:
f1_space,dummy,dummy2=SymbolicNumericIntegration.integrate(integrand,X; symbolic=true)
f2_space,dummy,dummy2=SymbolicNumericIntegration.integrate(f1_space,X; symbolic=true)

In [ ]:
f2_space

In [ ]:
@variables τ

F2_space(Δx,τ)

In [ ]:
F2_space(X,T)


In [ ]:
F2_space(X,T)

In [ ]:
#@variables PP[1:L₁ₗ+L₁ᵣ+1] MM[1:L₁ₗ+L₁ᵣ+1]
F2_space=expand(F2_space)
#f=substitute(F2_space,Dict([X=>Δx]))
#F2_space=X^2+X
#@variables xx
#F2_space=X^2*ρₘ[2]*Cᵘ[1,3]/Δx
#@variables y
#\f=substitute(F2_space,Dict([ρₘ=>PP,μₘ=>MM,X=>Δx]))
#f


In [ ]:
integrand_for_time=-1/Δx*substitute(F2_space,Dict([X=>Δx]))+(-1/Δx)*substitute(F2_space,Dict([X=>Δx]))

# Symbolic integral
Here we evaluate the binlinar integral operators step by step.

The goal is:
\begin{equation}
\int dt \psi^{(n)} \int dx \phi^{(m)} \left [ \rho (x) \frac{\partial^2}{\partial t^2} u(x,t) - \frac{\partial}{\partial x} \left ( \mu (x) \frac{\partial}{\partial x} u(x,t) \right ) \right ]
\end{equation}

So first 'integrand_before' means:
\begin{equation}
 f(x)=\left [ \rho (x) \frac{\partial^2}{\partial t^2} u(x,t) - \frac{\partial}{\partial x} \left ( \mu (x) \frac{\partial}{\partial x} u(x,t) \right ) \right ]
\end{equation}
and its integral over $x^1$ ('space_integrated') will be:
\begin{equation}
F^{(1)}(x)=\int dx  \left [ \rho (x) \frac{\partial^2}{\partial t^2} u(x,t) - \frac{\partial}{\partial x} \left ( \mu (x) \frac{\partial}{\partial x} u(x,t) \right ) \right ]
\end{equation}
...

The aim of this practice is to use the expression:
\begin{align}
\int_{y_1}^{y_2} dy f(y) w(y) = \left [F^{(1)}(y) w(y) \right ]_{y_1}^{y_2} -  \left [F^{(2)}(y) \frac{\partial w(y)}{\partial y} \right ]_{y_1}^{y_2} + \left [F^{(3)}(y) \frac{\partial^2 w(y)}{\partial y^2} \right ]_{y_1}^{y_2} - \cdots 
\end{align}

In [ ]:
# We evaluate the bilinear integral operators step by step
∂ₜ = Differential(t)
∂₁ = Differential(x¹)

integrand=[]

degree_of_weighting_function_in_terms_of_polynomial = 2 # if we use linear spline,  this should be 1

integrand_before=ρ(x¹-ξ¹[L₁ₗ+1])*(∂ₜ^2)(u(x¹-ξ¹[L₁ₗ+1] , t-τ[Lₜₗ+1]))-∂₁(μ(x¹-ξ¹[L₁ₗ+1])*∂₁(u(x¹-ξ¹[L₁ₗ+1] , t-τ[Lₜₗ+1])))|>expand_derivatives
integrand_before=substitute(integrand_before,Dict([Cᴾ=>newCᴾ, Cᴹ=>newCᴹ]))
integrand_before=simplify(integrand_before)
integrand=push!(integrand,integrand_before)
for iDegree in 1:degree_of_weighting_function_in_terms_of_polynomial
    #print(integrand_before)
    integrand_before=simplify(integrate(integrand_before,x¹))
    integrand=push!(integrand,integrand_before)
end


In [ ]:
integrand[3]

In [ ]:
# trial functions
function TruncatedSinc(x,scaling_factor=1)
    # if scaling_factor is 1, then the sinc function is scaled by π and has 0 every node=1
    #  (i.e. the sinc function is sin(π*x)/(π*x)
    # if we change the scaling_factor then the node interval is multiplied by the scaling_factor
    # if scaling_factor is 0, then the sinc function is unscaled (i.e. the sinc function is sin(x)/x)
    if scaling_factor == 0
        scaling_factor = π
    end 
    x=x/scaling_factor
    IfElse.ifelse(x==0,1,sin(π*x)/(π*x))
end

function Spline(x,j,k,nodes)
    IfElse.ifelse(k>0,
        IfElse.ifelse(j+1 <= length(nodes),
            (x-nodes[j])/(nodes[j+k]-nodes[j])*Spline(x,j,k-1,nodes),
            0
        )+
        IfElse.ifelse(j+k+1 <= length(nodes),
            (nodes[j+k+1]-x)/(nodes[j+k+1]-nodes[j+1])*Spline(x,j+1,k-1,nodes),
            0
        ),
        IfElse.ifelse(j>=1 && j+1 <=length(nodes) && nodes[j] <= x < nodes[j+1],
            1,
            0
        )
    )
    """
    if k>0 
        if j+1 <= length(nodes)
            X_local += (x-nodes[j])/(nodes[j+k]-nodes[j])*Spline(x,j,k-1,nodes) 
        end
        if j+k+1 <= length(nodes)
            X_local += X_local + (nodes[j+k+1]-x)/(nodes[j+k+1]-nodes[j+1])*Spline(x,j+1,k-1,nodes)
        end      
    elseif k==0
        if j>=1 && j+1 <=length(nodes) && nodes[j] <= x < nodes[j+1]
            X_local = 1.e0
        end
    end
    """
end

In [ ]:
ϕᵐ(x¹) = Spline(x¹,L₁ₗ+1,1,ξ¹) # x₁ direction linear spline function
ψⁿ(t) = Spline(t,Lₜₗ+1,1,τ)

In [ ]:
ϕᵐ(x¹)

In [ ]:
u(x¹-ξ¹[L₁ₗ+1] , t-τ[Lₜₗ+1])

In [ ]:
ρ(x¹-ξ¹[L₁ₗ+1])

In [ ]:
@variables x t Δx Δt f ρₒ μₒ uₒ
@syms ρ(x) μ(x)
∂ₜ = Differential(t)
∂ₓ = Differential(x)

Taylorₓ(f,order) = f+sum((∂ₓ^i)(f(x))*Δx^i/factorial(BigInt(i)) for i in 1:order)
Taylorₜ(f,order) = f+sum((∂ₜ^i)f*Δt^i/factorial(BigInt(i)) for i in 1:order)
Taylorₓₜ(f,order) = f+sum((∂ₓ^i)(f)*Δx^i/factorial(BigInt(i)) for i in 1:order) + sum((∂ₜ^i)(f)*Δt^i/factorial(BigInt(i)) for i in 1:order) +sum(sum((∂ₜ^k)((∂ₓ^i)(f))*Δx^i/factorial(BigInt(i))*Δt^k/factorial(BigInt(k)) for i in 1:order) for k in 1:order)
DerivativeₓTaylorₓ(f,order) = sum((∂ₓ^i)(f)*Δx^(i-1)/factorial(BigInt(i-1)) for i in 1:order)
DerivativeₜTaylorₜ(f,order) = sum((∂ₓ^i)(f)*Δt^(i-1)/factorial(BigInt(i-1)) for i in 1:order)
DerivativeₓTaylorₓₜ(f,order) = sum((∂ₓ^i)(f)*Δx^(i-1)/factorial(BigInt(i-1)) for i in 1:order) + +sum(sum((∂ₜ^k)((∂ₓ^i)(f))*Δx^(i-1)/factorial(BigInt(i-1))*Δt^k/factorial(BigInt(k)) for i in 1:order) for k in 1:order)
DerivativeₜTaylorₓₜ(f,order) = sum((∂ₜ^i)(f)*Δt^(i-1)/factorial(BigInt(i-1)) for i in 1:order) + +sum(sum((∂ₜ^k)((∂ₓ^i)(f))*Δx^i/factorial(BigInt(i-1))*Δt^(k-1)/factorial(BigInt(k-1)) for i in 1:order) for k in 1:order)

In [ ]:
@variables xₘ tₙ
u=substitute(Taylorₓₜ(uₒ,4),Dict(Δx=>x-xₘ, Δt=>t-tₙ))
ρ=substitute(Taylorₓ(ρ₀,4),Dict(Δx=>x-xₘ))
μ=substitute(Taylorₓ(μ₀,4),Dict(Δx=>x-xₘ))
u₁=substitute(DerivativeₓTaylorₓₜ(uₒ,4),Dict(Δx=>x-xₘ, Δt=>t-tₙ))


In [ ]:
@syms ρ(x,t) xₘ
tmpTaylor(f,order)=(∂ₓ^2)(f(x,t)) |>expand_derivatives
z(x,t)=tmpTaylor(ρ,2)
z(xₘ,t)


In [ ]:
Differential(x)(x^2) |>expand_derivatives